<a href="https://colab.research.google.com/github/pan7ae/laba_2_wiki/blob/master/laba_2_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import requests
from bs4 import BeautifulSoup
from typing import List
import time
from collections import deque

In [22]:
def send_request(url: str, rate_limit: int) -> str:
    """
    Отправка GET-запроса на URL-адрес

    :param url: URL-адрес страницы с Wikipedia
    :param rate_limit: Ограничение на количество запросов
    :return: Возвращает текст ответа с сервера
    """
    time.sleep(60 / rate_limit)
    response = requests.get(url)
    response.raise_for_status()
    return response.text

In [23]:
def get_wikipedia_links(html: requests.Response.text, base_url: str) -> List[str]:
    """
    Парсинг HTML-текста и возвращает список ссылок на другие статьи на сайте Wikipedia.

    :param html: Текст HTTP-запроса
    :param base_url: Базовая сслыка на Wikipedia
    :return: Список ссылок, ведущих на другие статьи Wikipedia
    """
    soup = BeautifulSoup(html, "html.parser")
    main_content = soup.find("div", {"id": "bodyContent"})
    all_links = main_content.find_all("a", href=True)

    wiki_links = []

    for link in all_links:
        href = link["href"]
        if href.startswith("/wiki/") and ":" not in href:
            full_url = base_url + href
            wiki_links.append(full_url)

    return wiki_links

In [24]:
def find_shortest_path(url_1: str, url_2: str, rate_limit: int) -> List[str] | None:
    """
    Используем очередь и поиск в ширину для обхода графа связей между страницами.

    :param url_1: URL-адрес страницы с Wikipedia
    :param url_2: URL-адрес страницы с Wikipedia
    :param rate_limit: Ограничение на количество запросов
    :return: Кратчайший путь в виде списка URL-адресов или None
    """
    base_url = "https://en.wikipedia.org"

    # Очередь путей, которые нужно исследовать
    queue = deque()

    # Множество посещенных адресов для избежания дублирования
    visited = set()

    queue.append(([url_1], 0))
    visited.add(url_1)

    while queue:
        path, depth = queue.popleft()

        if depth > 5:
            return None

        last_url = path[-1]
        html = send_request(last_url, rate_limit)
        wiki_links = get_wikipedia_links(html, base_url)

        for link in wiki_links:
            if link == url_2:
                path.append(link)
                return path

            if link not in visited:
                visited.add(link)
                new_path = path.copy()
                new_path.append(link)
                queue.append((new_path, depth + 1))

    return None

In [26]:
url1 = "https://en.wikipedia.org/wiki/Dietrich_Mateschitz"
url2 = "https://en.wikipedia.org/wiki/Lewis_Hamilton"
rate_limit = 10

shortest_path = find_shortest_path(url1, url2, rate_limit)
if shortest_path:
    print(" => ".join(shortest_path))
else:
    print("Цепочка переходов не найдена за 5 шагов.")

https://en.wikipedia.org/wiki/Dietrich_Mateschitz => https://en.wikipedia.org/wiki/List_of_Formula_One_World_Constructors%27_Champions => https://en.wikipedia.org/wiki/Lewis_Hamilton
